In [2]:
import os
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import pandas as pd
import random

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense


In [ ]:
datadir = 'Behavioural Cloning\Data Collected\driving_log.csv'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', -1)
data.head()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2120989838.py, line 1)